<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/sentence_seach_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Function to get sentence embeddings with pooling over the last hidden state
def get_sentence_embedding(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='tf', padding=True, truncation=True, max_length=512)

    # Get the BERT model outputs (hidden states)
    outputs = model(**inputs)

    # Last hidden states (batch_size, seq_length, hidden_size)
    last_hidden_state = outputs.last_hidden_state

    # Apply mean pooling over the sequence dimension
    pooled_embedding = tf.reduce_mean(last_hidden_state, axis=1)

    # print(pooled_embedding.shape, '\n')
    return pooled_embedding

In [21]:
documents = [
    "Machine learning enables AI.",
    "FAISS is a library for fast similarity search.",
    "Transformers provide state-of-the-art NLP models.",
    "Deep learning is a subset of machine learning.",
    "how are you ?"
]


doc_embeddings = np.vstack([get_sentence_embedding(doc) for doc in documents])

def search(query, top_k=2):
    """Find the most relevant documents using cosine similarity"""
    query_embedding = get_sentence_embedding([query])

    # Compute cosine similarity between query and document embeddings
    similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Get top-k most similar documents
    top_indices = np.argsort(similarities)[::-1][:top_k]

    results = [(documents[i], similarities[i]) for i in top_indices]
    return results


In [22]:
query = "How does deep learning work?"
results = search(query)

# Display results
for res, score in results:
    print(f"Similarity Score: {score:.4f}, Document: {res}")

Similarity Score: 0.7120, Document: Machine learning enables AI.
Similarity Score: 0.6804, Document: Deep learning is a subset of machine learning.
